# Quick test/overview

blah, blah, blah....

In [1]:
import potentials

## 0. Check loaded record styles

All records that the potentials package is designed to interact with are managed by potentials.recordmanager.  Calling the recordmanager's check_styles() method will provide a quick overview of the available styles.  All styles should pass import, but if one doesn't the most likely reasons are that either

- the record style requires additional Python package requirements beyond the base package, or
- the record style is in development and not fully ready for use yet.

__NOTE__: the potentials package is designed solely for interacting with records associated with the interatomic potentials listings found on the NIST Interatomic Potentials Repository.  Other related packages (will) expand the available record styles as is appropriate for what those packages are designed to represent and interact with:

- atomman adds record styles associated with crystal prototypes and crystal structures, and defect generation input parameter sets.
- iprPy adds record styles associated with property calculations performed for the various interatomic potentials.

In [2]:
potentials.recordmanager.check_styles()

Record styles that passed import:
- Citation
- Potential
- potential_LAMMPS
- potential_LAMMPS_KIM
Record styles that failed import:



## 1. Initializing potentials.Database

The potentials.Database object provides a single interface for interacting with data records stored in a local directory and/or in a remote CDCS database.  

For clarity, the initialization parameters of Database are divided based on the categories that they represent.

### 1.1. Initialization parameters for remote

These parameters control the settings for interactions with the remote database. For most users, only "remote" is important.  Otherwise, "username" and "password" only matter if you have an account for https://potentials.nist.gov/ with write access. The remaining parameters allow for a different CDCS database to be interfaced with should an alternate potentials host ever exist.

- __remote__  (*bool, optional*) Indicates if the load operations will download records from the remote database.  Default value is controlled by settings.  If False, then host, username, password, cert and verify values are ignored.
- __host__  (*str, optional*) Remote CDCS site to access.  Default value is 'https://potentials.nist.gov/'.
- __username__  (*str, optional*) User name to use to access the remote site.  Default value of '' will access the site as an anonymous visitor.
- __password__  (*str, optional*) Password associated with the given username.  Not needed for anonymous access.
- __cert__  (*str or tuple, optional*) File path(s) to certification file(s) if needed for host.
- __verify__  (*bool or str, optional*) Verification options.  If not given, will default to True if cert is given and False if cert is not given.

### 1.2. Initialization parameters for local 

These parameters control the settings for interactions with the local database.  It is highly recommended to use potentials.settings to specify the default settings once rather than giving the values everytime a database is initialized to avoid any record format conflicts.

- __local__  (*bool, optional*) Indicates if the load operations will check localpath for records. Default value is controlled by settings.  If False, then localpath, format and indent values are ignored.
- __localpath__  (*str, optional*) Path to the local directory to use for the local database location. If not given, will use the library_directory setting.
- __format__  (*str, optional*) The file format to use for saving records to the local location. Allowed values are 'json' or 'xml'.  If not given, will use the library_format setting.
- __indent__  (*str, optional*) The indentation style to use for saving records to the local database location.  If not given, will use the library_indent setting.

### 1.3. Initialization parameters associated with LAMMPS potentials

These parameters alter settings associated with how the LAMMPS potential records are retrieved and set up.

- __status__  (*str, list or None, optional*) Only potential_LAMMPS records with the given status(es) will be loaded.  Allowed values are 'active' (default), 'superseded', and 'retracted'.  If None is given, then all potentials will be loaded.
- __pot_dir_style__  (*str, optional*) Specifies how the pot_dir values will be set for the loaded lammps potentials.  Allowed values are 'working', 'id', and 'local'. 'working' will set all pot_dir = '', meaning parameter files are expected in the working directory when the potential is accessed. 'id' sets the pot_dir values to match the potential's id. 'local' sets the pot_dir values to the corresponding local database paths where the files are expected to be found.  Default value is controlled by settings.
- __kim_models__  (*str or list, optional*) Allows for the list of installed_kim_models to be explicitly given. Cannot be given with the other parameters.
- __kim_api_directory__  (*path-like object, optional*) The directory containing the kim api to use to build the list.
- __kim_models_file__  (*path-like object, optional*) The path to a whitespace-delimited file listing full kim ids.



In [3]:
potdb = potentials.Database(local=True, remote=True, format='json', indent=4)

## 2. Underlying database objects

When initializing a potentials.Database object, underlying database interfaces are created for the local and/or remote locations if the local and/or remote parameters are respectively True.  If either is False, then the associated interface will not be created and will be None.  

In [4]:
print(potdb.local_database)
print(potdb.remote_database)

database style local at C:\Users\lmh1\Documents\library
database style cdcs at https://potentials.nist.gov/


### 2.1. set_local_database() and set_remote_database()

These methods allow for the underlying database interfaces to be set or changed to an existing potentials.Database object after initialization.  The parameters for each set function correspond to the initialization parameters listed above associated with the two interfaces respectively (except for "local" and "remote"). Mostly, these are useful if remote or local was False during initialization and you would rather set the values on the existing object rather than re-initializing.

## 3. Generic record interaction methods

This section lists the Database methods associated with generic record interactions.  For citations, potentials and LAMMPS potentials, it is recommended to use the associated specific functions instead.  See the following set of Notebooks for more details on the specific methods.


### 3.1. get_records()

The get_records() method retrieves all matching records from the database.

- __style__  (*str, optional*) The record style to search. If not given, a prompt will ask for it.
- __name__  (*str or list, optional*) The name(s) of records to limit the search by.
- __local__  (*bool, optional*) Indicates if the local location is to be searched.  Default value matches the value set when the database was initialized.
- __remote__  (*bool, optional*) Indicates if the remote location is to be searched.  Default value matches the value set when the database was initialized.
- __verbose__  (*bool, optional*) If True, info messages will be printed during operations.  Default value is False.
- __return_df__  (*bool, optional*) If True, then the corresponding pandas.Dataframe of metadata will also be returned.
- __**kwargs__  (*any, optional*) Any extra keyword arguments supported by the record style.

In [5]:
# Find all citations from local and remote for author Mishin
records, df = potdb.get_records('Potential', return_df=True, verbose=True,
                                author='Mishin')

Found 25 matching Potential records in local library
Found 25 matching Potential records in remote library
 - 0 remote records are new


Setting return_df=True will return a table of the metadata fields associated with the retrieved records.

In [6]:
df

,name,key,id,recorddate,notes,fictional,elements,othername,modelname,citations,implementations
0,potential.1999--Mishin-Y-Farkas-D-Mehl-M-J-Pap...,0c2ddffb-e644-4e5c-8e53-9bf722bb5dee,1999--Mishin-Y-Farkas-D-Mehl-M-J-Papaconstanto...,2018-08-15,NaN,False,[Al],NaN,NaN,"[{'name': '10.1103_physrevb.59.3393', 'year': ...",[{'key': 'e107e2ce-beec-494e-b065-ff26f22eac15...
1,potential.1999--Mishin-Y-Farkas-D-Mehl-M-J-Pap...,81adb388-59eb-4f49-88bf-6f06a1343fae,1999--Mishin-Y-Farkas-D-Mehl-M-J-Papaconstanto...,2018-08-15,NaN,False,[Ni],NaN,NaN,"[{'name': '10.1103_physrevb.59.3393', 'year': ...",[{'key': '0b71eeb7-5e30-4206-bd59-6a259c57e3e4...
2,potential.2001--Mishin-Y-Mehl-M-J-Papaconstant...,a7910725-e80c-4a53-83e8-adfb5724dc9f,2001--Mishin-Y-Mehl-M-J-Papaconstantopoulos-D-...,2018-08-15,This listing is for the reference's EAM1 poten...,False,[Cu],NaN,1,"[{'name': '10.1103_physrevb.63.224106', 'year'...",[{'key': 'a063df15-8cb7-4773-8e50-79e6a889c27c...
3,potential.2001--Mishin-Y-Mehl-M-J-Papaconstant...,5a94a06a-22c7-41e0-8593-ee42d4728018,2001--Mishin-Y-Mehl-M-J-Papaconstantopoulos-D-...,2018-08-15,This listing is for the reference's EAM2 poten...,False,[Cu],NaN,2,"[{'name': '10.1103_physrevb.63.224106', 'year'...",[{'key': 'd6c12a91-1b3d-4cd5-8d90-ee0b2fcbf74b...
4,potential.2002--Mishin-Y-Mehl-M-J-Papaconstant...,cba1f03d-4e0b-4f08-8b52-0aff0a4224b3,2002--Mishin-Y-Mehl-M-J-Papaconstantopoulos-D-...,2018-08-15,"As described in the reference, this potential ...",False,"[Ni, Al]",NaN,NaN,"[{'name': '10.1103_physrevb.65.224114', 'year'...",[{'key': '1bda5691-8a4f-4597-ac00-a471898d27a6...
5,potential.2003--Zope-R-R-Mishin-Y--Al,36dbb1ec-c7f0-46a4-a501-1c518f777d5c,2003--Zope-R-R-Mishin-Y--Al,2018-08-15,NaN,False,[Al],NaN,NaN,"[{'name': '10.1103_physrevb.68.024102', 'year'...",[{'key': 'a6237bcc-a4be-497d-97ed-636f3999c541...
6,potential.2003--Zope-R-R-Mishin-Y--Ti-Al,274cbc15-b7c1-4701-a3e8-07b0b046ad7b,2003--Zope-R-R-Mishin-Y--Ti-Al,2018-08-15,NaN,False,"[Ti, Al]",NaN,NaN,"[{'name': '10.1103_physrevb.68.024102', 'year'...",[{'key': '4a23cdcd-0370-4702-99b9-8a5ff4824149...
7,potential.2004--Mishin-Y--Ni-Al,13051c1b-48f7-47f5-bad7-fc627fe10376,2004--Mishin-Y--Ni-Al,2018-08-15,NaN,False,"[Ni, Al]",NaN,NaN,"[{'name': '10.1016_j.actamat.2003.11.026', 'ye...",[{'key': '239e4f9a-145d-4c93-a967-73fb3ba28fea...
8,potential.2005--Mishin-Y-Mehl-M-J-Papaconstant...,76452c02-4b98-4632-bce7-93fec036f745,2005--Mishin-Y-Mehl-M-J-Papaconstantopoulos-D-...,2018-08-15,NaN,False,"[Fe, Ni]",NaN,NaN,"[{'name': '10.1016_j.actamat.2005.05.001', 'ye...",[{'key': '4227d6c9-63ed-4c2e-9740-2215bdbd89b4...
9,potential.2006--Chamati-H-Papanicolaou-N-I-Mis...,3837b6da-05c4-4148-97cd-90b26a2092df,2006--Chamati-H-Papanicolaou-N-I-Mishin-Y-Papa...,2018-08-15,NaN,False,[Fe],NaN,NaN,"[{'name': '10.1016_j.susc.2006.02.010', 'year'...",[{'key': '9e16d628-87cc-49a0-8ce8-a23097f798e5...


The df is useful for further parsing and sorting of the records after they have been downloaded.  For instance, here the listings are further filtered to only those containing Co.

In [7]:
def has_element(series, element):
    return element in series.elements

df[df.apply(has_element, axis=1, args=['Co'])]

,name,key,id,recorddate,notes,fictional,elements,othername,modelname,citations,implementations
16,potential.2012--Purja-Pun-G-P-Mishin-Y--Co,a51fd82a-2bb5-4d30-9663-5f730c6fab40,2012--Purja-Pun-G-P-Mishin-Y--Co,2018-08-15,NaN,False,[Co],NaN,NaN,"[{'name': '10.1103_physrevb.86.134116', 'year'...",[{'key': 'dc7b14cc-ed13-4332-94de-d91c0f839b98...
19,potential.2015--Purja-Pun-G-P-Yamakov-V-Mishin...,71b03297-b66d-4efd-9502-02cf83d9dc38,2015--Purja-Pun-G-P-Yamakov-V-Mishin-Y--Al-Co,2018-08-15,The reference information was updated on 26 Au...,False,"[Al, Co]",NaN,NaN,"[{'name': '10.1088_0965-0393_23_6_065006', 'ye...",[{'key': 'ef327c3b-3760-4cbd-b9a5-5a7f5c66bebe...
20,potential.2015--Purja-Pun-G-P-Yamakov-V-Mishin...,0f6e3cef-c588-4dcb-a0a6-66878f64ac50,2015--Purja-Pun-G-P-Yamakov-V-Mishin-Y--Ni-Al-Co,2018-08-15,The reference information was updated on 26 Au...,False,"[Ni, Al, Co]",NaN,NaN,"[{'name': '10.1088_0965-0393_23_6_065006', 'ye...",[{'key': 'ec7cf9ba-a69b-48b2-9142-d89debe5e3ee...
21,potential.2015--Purja-Pun-G-P-Yamakov-V-Mishin...,20fa46b5-6b6f-4e6e-9400-66edb643c679,2015--Purja-Pun-G-P-Yamakov-V-Mishin-Y--Ni-Co,2018-08-15,The reference information was updated on 26 Au...,False,"[Ni, Co]",NaN,NaN,"[{'name': '10.1088_0965-0393_23_6_065006', 'ye...",[{'key': 'beb35d7a-eb97-4c9d-b141-2b1b83879a29...


As the returned records are part of a numpy array, the same conditional pandas indexing can be applied directly to the list of records. 

In [8]:
co_records = records[df.apply(has_element, axis=1, args=['Co'])]
for record in co_records:
    print(record.name)

potential.2012--Purja-Pun-G-P-Mishin-Y--Co
potential.2015--Purja-Pun-G-P-Yamakov-V-Mishin-Y--Al-Co
potential.2015--Purja-Pun-G-P-Yamakov-V-Mishin-Y--Ni-Al-Co
potential.2015--Purja-Pun-G-P-Yamakov-V-Mishin-Y--Ni-Co


### 3.2. get_record()

The get_record() method retrieves a single matching record from the database and throws an error if exactly one match is not found.  If a match is found in local, then the remote will not be searched.

- __style__  (*str, optional*) The record style to search. If not given, a prompt will ask for it.
- __name__  (*str or list, optional*) The name(s) of records to limit the search by.
- __local__  (*bool, optional*) Indicates if the local location is to be searched.  Default value matches the value set when the database was initialized.
- __remote__  (*bool, optional*) Indicates if the remote location is to be searched.  Default value matches the value set when the database was initialized.
- __verbose__  (*bool, optional*) If True, info messages will be printed during operations.  Default value is False.
- __**kwargs__  (*any, optional*) Any extra keyword arguments supported by the record style.

In [9]:
record = potdb.get_record('Potential', id='2015--Purja-Pun-G-P-Yamakov-V-Mishin-Y--Ni-Al-Co', verbose=True)
print(record.id)

Matching record retrieved from local
2015--Purja-Pun-G-P-Yamakov-V-Mishin-Y--Ni-Al-Co


In [10]:
record = potdb.get_record('Potential', id='2015--Purja-Pun-G-P-Yamakov-V-Mishin-Y--Ni-Al-Co', local=False, verbose=True)
print(record.id)

Matching record retrieved from remote
2015--Purja-Pun-G-P-Yamakov-V-Mishin-Y--Ni-Al-Co


### 3.3. remote_query()

The remote_query() method allows for custom queries of the remote database rather than using the built-in record-style-specific queries.  Note that these query options are specific to CDCS-style databases and therefore cannot search local records. 
    
- __style__  (*str, optional*) The record style to search. If not given, a prompt will ask for it.
- __name__  (*str or list, optional*) The name(s) of records to limit the search by.
- __return_df__  (*bool, optional*) If True, then the corresponding pandas.Dataframe of metadata will also be returned
- __query__  (*dict, optional*) A custom-built CDCS Mongo-style query to use for the record search. Cannot be given with keyword.
- __keyword__  (*str, optional*) Allows for a search of records whose contents contain a keyword. Cannot be given with query.

Use keyword content search to find all potentials records that contain "radiation"

In [11]:
records = potdb.remote_query('Potential', keyword='radiation')
print(len(records), 'records contain "radiation"')

41 records contain "radiation"


Info for custom query fields.  CDCS databases take Mongo-like find queries.  They differ from pure Mongo queries in that 

- the paths start at the record root, and
- projections are not allowed, i.e. only full records can be retrieved.

In [12]:
# Print the first bit of a record in json format
print(records[0].build_model().json(indent=4)[:300])

{
    "interatomic-potential": {
        "key": "dc4149ce-3592-4131-8683-ecf654d5a519",
        "id": "1987--Ackland-G-J-Tichy-G-Vitek-V-Finnis-M-W--Ag",
        "record-version": "2018-08-15",
        "description": {
            "citation": {
                "document-type": "journal",
           


In [13]:
# build an example custom query for key
query = {"interatomic-potential.key": "dc4149ce-3592-4131-8683-ecf654d5a519"}

records = potdb.remote_query('Potential', query=query)
print(records[0].id)

1987--Ackland-G-J-Tichy-G-Vitek-V-Finnis-M-W--Ag


### 3.4. download_records()

In [1]:
import potentials

In [2]:
potdb = potentials.Database(local=True, remote=True, format='json', indent=4)

In [4]:
potdb.download_records('Citation', overwrite=True, verbose=True)

Found 355 matching Citation records in remote library
354 new records added to local
1 existing records changed in local


### 3.5. upload_record()

### 3.6. delete_record()